In [1]:
!pip install numpy==1.26.4


Defaulting to user installation because normal site-packages is not writeable


In [2]:
! "C:\Program Files\Python310\python.exe" -m pip install ipykernel
! "C:\Program Files\Python310\python.exe" -m ipykernel install --user --name py310 --display-name "Python 3.10 (TensorFlow)"


Defaulting to user installation because normal site-packages is not writeable


Installed kernelspec py310 in C:\Users\Abinaya\AppData\Roaming\jupyter\kernels\py310


In [3]:
!pip install tensorflow==2.10.1

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import tensorflow as tf
import numpy as np
import cv2
import os
import time


In [5]:
!where pip
!pip show tensorflow


C:\Program Files\Python310\Scripts\pip.exe
C:\Users\Abinaya\AppData\Local\Programs\Python\Python36\Scripts\pip.exe
C:\Users\Abinaya\AppData\Local\Programs\Python\Python313\Scripts\pip.exe
Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\abinaya\appdata\roaming\python\python310\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [6]:
import sys
print(sys.executable)


C:\Program Files\Python310\python.exe


In [7]:
!{sys.executable} -m pip install tensorflow


'C:\Program' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
import tensorflow as tf
model_dir = r"C:\Users\Abinaya\Downloads\TrafficSignDetection-main\TrafficSignDetection-main\saved_models\efficientdet_d1_coco17_tpu-32\saved_model"
detect_fn = tf.saved_model.load(model_dir)

In [14]:
import tensorflow as tf
classifier_model_path = r"C:\Users\Abinaya\Downloads\Traffic-Sign-Classification-master\Traffic-Sign-Classification-master\my_model.h5"
classifier = tf.keras.models.load_model(classifier_model_path)

In [15]:
detect_fn.summary()

AttributeError: '_UserObject' object has no attribute 'summary'

In [17]:
classifier.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 28, 28, 60)        1560      
                                                                 
 conv2d_18 (Conv2D)          (None, 24, 24, 60)        90060     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 12, 12, 60)       0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 10, 10, 30)        16230     
                                                                 
 conv2d_20 (Conv2D)          (None, 8, 8, 30)          8130      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 4, 4, 30)         0         
 g2D)                                                 

In [12]:
classifier_classes=[
    'Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)',
    'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (10km/h)',
    'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)',
    'No passing', 'No passing for vehicles over 3.5 metric tons',
    'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop',
    'No vehicles', 'Vehicles over 3.5 metric tons prohibited', 'No entry',
    'General caution', 'Dangerous curve to the left', 'Dangerous curve to the right',
    'Double curve', 'Bumpy Road', 'Slippery road', 'Road narrows on the right',
    'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing',
    'Beware of ice/snow', 'Wild animals crossing', 'End of all speed and passing limits',
    'Turn right ahead', 'Turn left ahead', 'Ahead only', 'Go straight or right',
    'Go straight or left', 'Keep right', 'Keep left', 'Roundabout mandatory',
    'End of no passing', 'End of no passing by vehicles over 3.5 metric tons'
]

In [24]:
detection_score_threshold = 0.2
classification_score_threshold = 0.3

In [17]:
input_video_path = r"C:\PROJECTS\TrafficSignIdentification\drivingdata\outsidevideo2cropped.mp4"
output_video_path = r"C:\PROJECTS\TrafficSignIdentification\output\output_outsidevideo2cropped(detectron+roboflow).mp4"



In [18]:
def preprocess_for_classifier(cropped_img):
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(gray, (32, 32))
    img_normalized = img_resized / 255.0
    return np.expand_dims(img_normalized[..., np.newaxis], axis=0)



In [19]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf


In [20]:
def run_inference_on_image(image):
    image_np_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_np_rgb)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    return detections


In [21]:
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    raise IOError(f"Cannot open video file: {input_video_path}")

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

total_start_time = time.time()
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_start_time = time.time()
    detections = run_inference_on_image(frame)

    boxes = detections['detection_boxes'].numpy()[0]
    scores = detections['detection_scores'].numpy()[0]

    for box, score in zip(boxes, scores):
        if score < detection_score_threshold:
            continue

        ymin, xmin, ymax, xmax = box
        left = int(xmin * width)
        right = int(xmax * width)
        top = int(ymin * height)
        bottom = int(ymax * height)

        cropped_img = frame[top:bottom, left:right]
        if cropped_img.size == 0:
            continue

        input_for_cls = preprocess_for_classifier(cropped_img)
        preds = classifier.predict(input_for_cls)
        class_id = np.argmax(preds)
        cls_score = preds[0][class_id]

        if cls_score < classification_score_threshold:
            label_text = "Unknown"
        else:
            label_text = f"{classifier_classes[class_id]} ({cls_score:.2f})"

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, label_text, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    video_writer.write(frame)
    frame_end_time = time.time()
    print(f"Processed frame {frame_count + 1} in {frame_end_time - frame_start_time:.3f} seconds")
    frame_count += 1

# Release resources
cap.release()
video_writer.release()

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"\nProcessed {frame_count} frames in {total_duration:.2f} seconds")
print(f"Average time per frame: {total_duration / frame_count:.2f} seconds")
print(f"Output video saved to: {output_video_path}")

Processed frame 1 in 1.102 seconds
Processed frame 2 in 1.115 seconds
Processed frame 3 in 1.236 seconds
Processed frame 4 in 1.172 seconds
Processed frame 5 in 0.984 seconds
Processed frame 6 in 1.003 seconds
1/1 [==============================] - 0s 61ms/step
Processed frame 7 in 1.193 seconds


KeyboardInterrupt: 

In [ ]:
#loading model for classifier
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense

base_model = load_model("my_model.h5")

# Remove the last layer
x = base_model.layers[-2].output

# Add new output layer with more classes (e.g., 80 instead of 43)
num_new_classes = 80
new_output = Dense(num_new_classes, activation='softmax')(x)

# Create new model
new_model = Model(inputs=base_model.input, outputs=new_output)


In [ ]:
#retraining only the new model
for layer in new_model.layers[:-1]:
    layer.trainable = False

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Use your new training data with 80 classes
new_model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))


In [25]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf

# Define paths
input_folder = r"C:\PROJECTS\TrafficSignIdentification\drivingdata\New folder"
output_folder = r"C:\PROJECTS\TrafficSignIdentification\output\imagesthreshold=0.2"
os.makedirs(output_folder, exist_ok=True)

# Detection threshold values
detection_score_threshold = 0.5
classification_score_threshold = 0.5

def preprocess_for_classifier(cropped_img):
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(gray, (32, 32))
    img_normalized = img_resized / 255.0
    return np.expand_dims(img_normalized[..., np.newaxis], axis=0)

def run_inference_on_image(image):
    image_np_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_np_rgb)[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    return detections

# Process each image in the input folder
image_filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

total_start_time = time.time()
processed_count = 0

for filename in image_filenames:
    image_path = os.path.join(input_folder, filename)
    frame = cv2.imread(image_path)

    detections = run_inference_on_image(frame)
    boxes = detections['detection_boxes'].numpy()[0]
    scores = detections['detection_scores'].numpy()[0]

    for box, score in zip(boxes, scores):
        if score < detection_score_threshold:
            continue

        ymin, xmin, ymax, xmax = box
        h, w, _ = frame.shape
        left = int(xmin * w)
        right = int(xmax * w)
        top = int(ymin * h)
        bottom = int(ymax * h)

        cropped_img = frame[top:bottom, left:right]
        if cropped_img.size == 0:
            continue

        input_for_cls = preprocess_for_classifier(cropped_img)
        preds = classifier.predict(input_for_cls)
        class_id = np.argmax(preds)
        cls_score = preds[0][class_id]

        if cls_score < classification_score_threshold:
            label_text = "Unknown"
        else:
            label_text = f"{classifier_classes[class_id]} ({cls_score:.2f})"

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, label_text, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, frame)
    print(f"Processed and saved: {filename}")
    processed_count += 1

total_end_time = time.time()
print(f"\nProcessed {processed_count} images in {total_end_time - total_start_time:.2f} seconds.")


Processed and saved: IMG-20250620-WA0001.jpg
Processed and saved: IMG-20250620-WA0002.jpg
Processed and saved: IMG-20250620-WA0003.jpg
1/1 [==============================] - 0s 34ms/step
Processed and saved: IMG-20250620-WA0004.jpg
Processed and saved: IMG-20250620-WA0005.jpg
1/1 [==============================] - 0s 47ms/step
Processed and saved: IMG-20250620-WA0006.jpg
Processed and saved: IMG-20250620-WA0007.jpg
1/1 [==============================] - 0s 57ms/step
Processed and saved: IMG-20250620-WA0008.jpg
1/1 [==============================] - 0s 61ms/step
Processed and saved: IMG-20250620-WA0009.jpg
Processed and saved: IMG-20250620-WA0010.jpg
1/1 [==============================] - 0s 47ms/step
Processed and saved: IMG-20250620-WA0011.jpg
Processed and saved: IMG-20250620-WA0012.jpg
Processed and saved: IMG-20250620-WA0013.jpg
1/1 [==============================] - 0s 62ms/step
Processed and saved: IMG-20250620-WA0014.jpg
Processed and saved: IMG-20250620-WA0015.jpg
Processed and